## This is my work template for building ML models

The typical ML workflow as follows:
1. Define the Problem
2. Data Ingestion
3. Data Splitting: The training set is used to train the model, the validation set is used to tune hyperparameters, and the test set is used to evaluate the model's performance.
4. Data Exploration
5. Data Preprocessing: Missing data, imbalance, outliers, data transformation (e.g., normalization, encoding categorical variables).
6. Feature Engineering: Engineer new features from the existing data or perform dimensionality reduction techniques like PCA if needed
7. Feature Selection
8. Modelling
9. Hyperparameter Tuning
10. Evaluating the model
11. Iteration

# Uncle Steve's Amazing Do-All function

It slices! It dices!

In order to streamline the evaluation of each dataset, let's create an function that takes in a dataset, the name of the target column, and the name of any columsn to drop (because that's decided by the human), and then automate the rest:

- Converting datatypes of the target column if necessary
- OHE any categorical features
- Splitting data into training and testing
- Training and evaluating all the models/ensembles
- Returning a list of the performance of all the models

In [1]:
import datetime
print(datetime.datetime.now())

2023-10-08 17:56:46.008497


In [2]:
import pandas as pd
pd.show_versions(as_json=False)

import sklearn
sklearn.__version__


INSTALLED VERSIONS
------------------
commit           : 8dab54d6573f7186ff0c3b6364d5e4dd635ff3e7
python           : 3.8.10.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.22631
machine          : AMD64
processor        : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_Canada.1252

pandas           : 1.5.2
numpy            : 1.24.1
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 56.0.0
pip              : 23.2.1
Cython           : 0.29.14
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.1.2
IPython          : 8.8.0
pandas_datareader: None
bs4              : None
bottleneck       : None
brotli     

'1.2.0'

In [6]:
#!apt-get install swig -y
#!pip install Cython numpy
# sometimes you have to run the next command twice on colab
# I haven't figured out why
#!pip install auto-sklearn

In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.model_selection import train_test_split

import itertools

import scipy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_hist_gradient_boosting  # noqa

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

#import autosklearn.classification

import time

# Helper function
def do_all_for_dataset(dataset_name, df, target_col, drop_cols=[]):

    # If target_col is an object, convert to numbers
    if df[target_col].dtype == 'object':
      df[target_col] =  df[target_col].astype('category').cat.codes

    # OHE all categorical columns
    cat_cols = list(df.select_dtypes(include=['object']).columns) 
    if target_col in cat_cols: cat_cols.remove(targe_col)
    if len(cat_cols) > 0:
      df = pd.concat([df,pd.get_dummies(df[cat_cols])],axis=1)

    # Split into X and y
    X = df.drop(drop_cols + cat_cols + [target_col], axis=1)
    y = df[target_col]

    # Split into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    print('Y (train) counts:')
    print(y_train.value_counts())
    print('Y (test) counts:')
    print(y_test.value_counts())
    
    nb = GaussianNB()   
    lr = LogisticRegression(random_state=42, solver='lbfgs', max_iter=5000)
    dt = DecisionTreeClassifier(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=7)

    rf = RandomForestClassifier(random_state=42, n_estimators=200)
    ada = AdaBoostClassifier(random_state=42, n_estimators=200)

    # scorer = autosklearn.metrics.make_scorer(
    #     'f1_score',
    #     sklearn.metrics.f1_score
    # )    
    # automl = autosklearn.classification.AutoSklearnClassifier(
    #       time_left_for_this_task=100, # run auto-sklearn for at most X secs
    #       per_run_time_limit=15, # spend at most 60 sec for each model training
    #       metric=scorer
    #       )


    est_list = [('DT', dt), ('LR', lr), ('NB', nb), ('RF', rf), ('ADA', ada)]
       
    dict_classifiers = {
        "LR": lr, 
        "NB": nb,
        "DT": dt,
        "KNN": knn,
        "Voting": VotingClassifier(estimators = est_list, voting='soft'),
        "Bagging": BaggingClassifier(DecisionTreeClassifier(), n_estimators=200, random_state=42),
        "RF": rf,
        "ExtraTrees": ExtraTreesClassifier(random_state=42, n_estimators=200),
        "Adaboost": ada,
        "GBC": GradientBoostingClassifier(random_state=42, n_estimators=200),
        "Stacking": StackingClassifier(estimators=est_list, final_estimator=LogisticRegression()),
        #"automl": automl,
    }
    
    model_results = list()
    
    for model_name, model in dict_classifiers.items():
        start = time.time()
        y_pred = model.fit(X_train, y_train).predict(X_test)
        end = time.time()
        total = end - start
        
        accuracy       = accuracy_score(y_test, y_pred)
        f1             = f1_score(y_test, y_pred)
        recall         = recall_score(y_test, y_pred)
        precision      = precision_score(y_test, y_pred)
        roc_auc        = roc_auc_score(y_test, y_pred)
    
        df = pd.DataFrame({"Dataset"   : [dataset_name],
                           "Method"    : [model_name],
                           "Time"      : [total],
                           "Accuracy"  : [accuracy],
                           "Recall"    : [recall],
                           "Precision" : [precision],
                           "F1"        : [f1],
                           "AUC"       : [roc_auc],
                          })
        model_results.append(df)
   

    dataset_results = pd.concat([m for m in model_results], axis = 0).reset_index()

    dataset_results = dataset_results.drop(columns = "index",axis =1)
    dataset_results = dataset_results.sort_values(by=['F1'], ascending=False)
    dataset_results['Rank'] = range(1, len(dataset_results)+1)
    
    return dataset_results

In [15]:
# German Credit Example
df = pd.read_csv('https://raw.githubusercontent.com/stepthom/869_course/main/data/GermanCredit.csv')
r = do_all_for_dataset('GermanCredit', df, target_col='Class', drop_cols=[])
r

Y (train) counts:
1    560
0    240
Name: Class, dtype: int64
Y (test) counts:
1    140
0     60
Name: Class, dtype: int64


,Dataset,Method,Time,Accuracy,Recall,Precision,F1,AUC,Rank
9,GermanCredit,GBC,0.383559,0.760,0.857143,0.810811,0.833333,0.695238,1
7,GermanCredit,ExtraTrees,0.350254,0.745,0.857143,0.794702,0.824742,0.670238,2
6,GermanCredit,RF,0.382685,0.735,0.885714,0.770186,0.823920,0.634524,3
10,GermanCredit,Stacking,5.254151,0.730,0.835714,0.790541,0.812500,0.659524,4
5,GermanCredit,Bagging,0.917340,0.720,0.835714,0.780000,0.806897,0.642857,5
3,GermanCredit,KNN,0.015943,0.690,0.850000,0.743750,0.793333,0.583333,6
4,GermanCredit,Voting,0.913409,0.710,0.792857,0.792857,0.792857,0.654762,7
0,GermanCredit,LR,0.166558,0.700,0.792857,0.781690,0.787234,0.638095,8
8,GermanCredit,Adaboost,0.378950,0.695,0.778571,0.784173,0.781362,0.639286,9
2,GermanCredit,DT,0.003514,0.675,0.742857,0.781955,0.761905,0.629762,10


In [21]:
!pip install h2o
import h2o
h2o.init()

  Using cached h2o-3.42.0.4.tar.gz (240.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for h2o: filename=h2o-3.42.0.4-py2.py3-none-any.whl size=240946607 sha256=a554c7e841c73447fa58f42df51ba9458116ddc3b77039b07ade3c79bab80a4f
  Stored in directory: c:\users\james\appdata\local\pip\cache\wheels\ab\69\4a\df40b3ab6417eab057d3c36d70520eb99f363d704a688a7d3a
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\james\AppData\Local\Temp\tmpu_79qlij
  JVM stdout: C:\Users\james\AppData\Local\Temp\tmpu_79qlij\h2o_james_started_from_python.out
  JVM st

H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_james_4wy7j0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [22]:
data = h2o.import_file('https://raw.githubusercontent.com/stepthom/869_course/main/data/GermanCredit.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [26]:
data.head()

Duration,Amount,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance,Telephone,ForeignWorker,Class,CheckingAccountStatus.lt.0,CheckingAccountStatus.0.to.200,CheckingAccountStatus.gt.200,CheckingAccountStatus.none,CreditHistory.NoCredit.AllPaid,CreditHistory.ThisBank.AllPaid,CreditHistory.PaidDuly,CreditHistory.Delay,CreditHistory.Critical,Purpose.NewCar,Purpose.UsedCar,Purpose.Furniture.Equipment,Purpose.Radio.Television,Purpose.DomesticAppliance,Purpose.Repairs,Purpose.Education,Purpose.Vacation,Purpose.Retraining,Purpose.Business,Purpose.Other,SavingsAccountBonds.lt.100,SavingsAccountBonds.100.to.500,SavingsAccountBonds.500.to.1000,SavingsAccountBonds.gt.1000,SavingsAccountBonds.Unknown,EmploymentDuration.lt.1,EmploymentDuration.1.to.4,EmploymentDuration.4.to.7,EmploymentDuration.gt.7,EmploymentDuration.Unemployed,Personal.Male.Divorced.Seperated,Personal.Female.NotSingle,Personal.Male.Single,Personal.Male.Married.Widowed,Personal.Female.Single,OtherDebtorsGuarantors.None,OtherDebtorsGuarantors.CoApplicant,OtherDebtorsGuarantors.Guarantor,Property.RealEstate,Property.Insurance,Property.CarOther,Property.Unknown,OtherInstallmentPlans.Bank,OtherInstallmentPlans.Stores,OtherInstallmentPlans.None,Housing.Rent,Housing.Own,Housing.ForFree,Job.UnemployedUnskilled,Job.UnskilledResident,Job.SkilledEmployee,Job.Management.SelfEmp.HighlyQualified
6,1169,4,4,67,2,1,0,1,Good,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0
48,5951,2,2,22,1,1,1,1,Bad,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0
12,2096,2,3,49,1,2,1,1,Good,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0
42,7882,2,4,45,1,2,1,1,Good,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0
24,4870,3,4,53,2,2,1,1,Bad,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
36,9055,2,4,35,1,2,0,1,Good,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0
24,2835,3,4,53,1,1,1,1,Good,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0
36,6948,2,2,35,1,1,0,1,Good,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1
12,3059,2,4,61,1,1,1,1,Good,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0
30,5234,4,2,28,2,1,1,1,Bad,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1


In [27]:
#Split Data:
train, test = data.split_frame(ratios=[0.8], seed=42)
#Run AutoML:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_runtime_secs=600, seed=42)
aml.train(y="Class", training_frame=train)
#View AutoML Leaderboard:
lb = aml.leaderboard
print(lb)

AutoML progress: |
19:51:50.783: AutoML: XGBoost is not available; skipping it.
19:51:50.843: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]

█
19:51:52.279: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]
19:51:53.831: _train param, Dropping unused columns: [Personal.Female.Single, Purpose.Vacation]

█
19:51:54.232: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]

█
19:51:56.226: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]

██
19:51:57.281: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]
19:51:58.114: _train param, Dropping bad and constant columns: [Personal.Female.Single, Purpose.Vacation]

█
19:51:59.31: _train param, Dropping unused columns: [Personal.Female.Single, Purpose.Vacation]
19:51:59.263: _train param, Dropping unused columns: [Personal.Female.Single,

key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/5
# GBM base models (used / total),1/1
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),1/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_BestOfFamily_7_AutoML_1_20231008_195150  0.811108   0.471342  0.905776                0.337571  0.393697  0.154997
StackedEnsemble_BestOfFamily_5_AutoML_1_20231008_195150  0.811066   0.471253  0.905647                0.307998  0.393663  0.15497
StackedEnsemble_BestOfFamily_4_AutoML_1_20231008_195150  0.806986   0.475622  0.903333                0.311574  0.395665  0.156551
StackedEnsemble_AllModels_3_AutoML_1_20231008_195150     0.806513   0.476417  0.900113                0.34262   0.394858  0.155912
StackedEnsemble_AllModels_6_AutoML_1_20231008_195150     0.805355   0.476908  0.899479                0.310382  0.395139  0.156135
GBM_grid_1_AutoML_1_20231008_195150_model_12             0.801134   0.481867  0.897036                0.35466   0.397795  0.158241
StackedEnsemble_BestOfFamily_3_AutoML_1_20231008_195150  0.799622   0.481962  0.8976

In [28]:
#Make Predictions:
leader_model = aml.leader
predictions = leader_model.predict(test)

#Evaluate Model
leader_model.model_performance(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.18118958542255437
RMSE: 0.42566369991174297
LogLoss: 0.5337682340385339
AUC: 0.754731182795699
AUCPR: 0.885031423037741
Gini: 0.5094623655913979
Null degrees of freedom: 211
Residual degrees of freedom: 207
Null deviance: 256.3301430965229
Residual deviance: 226.3177312323384
AIC: 236.3177312323384

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1272298171147331
       Bad    Good    Error    Rate
-----  -----  ------  -------  ------------
Bad    4      58      0.9355   (58.0/62.0)
Good   0      150     0        (0.0/150.0)
Total  4      208     0.2736   (58.0/212.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.12723      0.837989  207
max f2                       0.12723      0.928218  207
max f0point5                 0.725869     0.833333  104
max accuracy                 0.528623     0.740566  152
max precision                0.998368     1         0
max recall                   0.12723      1         207
max specificity              0.998368     1         0
max absolute_mcc             0.725869     0.429474  104
max min_per_class_accuracy   0.666338     0.693333  121
max mean_per_class_accuracy  0.725869     0.736022  104
max tns                      0.998368     62        0
max fns                      0.998368     149       0
max fps                      0.0592125    62        211
max tps                      0.12723      150       207
max tnr                      0.998368     1         0
max fnr                      0.998368     0.993333  0
max fpr                      0.0592125    1         211
max tpr                      0.12723      1         207

Gains/Lift Table: Avg response rate: 70.75 %, avg score: 66.65 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0141509                   0.997886           1.41333   1.41333            1                0.998118  1                           0.998118            0.02            0.02                       41.3333   41.3333            0.02
2        0.0235849                   0.98714            1.41333   1.41333            1                0.992497  1                           0.99587             0.0133333       0.0333333                  41.3333   41.3333            0.0333333
3        0.0330189                   0.978752           1.41333   1.41333            1                0.983269  1                           0.99227             0.0133333       0.0466667                  41.3333   41.3333            0.0466667
4        0.0424528                   0.975218           1.41333   1.41333            1                0.976278  1                           0.988716            0.0133333       0.06                       41.3333   41.3333            0.06
5        0.0518868                   0.973153           1.41333   1.41333            1                0.974251  1                           0.986086            0.0133333       0.0733333                  41.3333   41.3333            0.0733333
6        0.103774                    0.944251           1.41333   1.41333            1                0.955263  1                           0.970674            0.0733333       0.146667                   41.3333   41.3333            0.146667
7        0.150943                    0.930803           1.272     1.36917            0.9              0.936419  0.96875                     0.95997      

In [29]:
#Shut Down H2O Cluster:
h2o.cluster().shutdown()

H2O session _sid_a6ae closed.
